In [1]:
from pyspark import SparkConf,SparkContext
from pyspark import sql
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()
sc = spark.sparkContext
 
jdbcDF = spark.read.format("jdbc")\
    .option("driver","com.mysql.jdbc.Driver")\
    .option("url","jdbc:mysql://139.155.70.177:3306/Gadaite")\
    .option("dbtable","audi")\
    .option("user","root")\
    .option("password","zzjz123")\
    .load()
jdbcDF.createOrReplaceTempView("temptable")

In [2]:
#查询最大值，最小值，平均值，和，行数
spark.sql("""
    select sum(price) as price 
    from temptable
    """).show()

+---------+
|    price|
+---------+
|244261836|
+---------+



In [3]:
spark.sql("""
    select 
    min(price) as minprice,
    max(price) as maxprice 
    from temptable
    """).show()

+--------+--------+
|minprice|maxprice|
+--------+--------+
|    1490|  145000|
+--------+--------+



In [4]:
spark.sql("""
    select 
    count(*) from temptable
    """).show()

+--------+
|count(1)|
+--------+
|   10668|
+--------+



In [5]:
#分组group by
spark.sql("""
    select 
    model,count(*) as count 
    from temptable group by model
    """).show()

+-----+-----+
|model|count|
+-----+-----+
|   A7|  122|
|  SQ7|    8|
|   A4| 1381|
|  RS3|   33|
|   S3|   18|
|  RS5|   29|
|  RS6|   39|
|  SQ5|   16|
|   S4|   12|
|   Q8|   69|
|  RS4|   31|
|   A1| 1347|
|   A3| 1929|
|   Q2|  822|
|  RS7|    1|
|   Q7|  397|
|   Q3| 1417|
|   R8|   28|
|   S5|    3|
|   Q5|  877|
+-----+-----+
only showing top 20 rows



In [6]:
#having过滤,此处使用where会报错
spark.sql("""
    select model,count(*) as count 
    from temptable 
    group by model 
    having count(*) >= 50
    """).show()

+-----+-----+
|model|count|
+-----+-----+
|   A7|  122|
|   A4| 1381|
|   Q8|   69|
|   A1| 1347|
|   A3| 1929|
|   Q2|  822|
|   Q7|  397|
|   Q3| 1417|
|   Q5|  877|
|   A5|  882|
|   TT|  336|
|   A8|  118|
|   A6|  748|
+-----+-----+



In [7]:
#having与where的同时使用,
#having相对于分组而言，而where相对于行而言
#如果指定用group，大多数数据库是等价的
spark.sql("""
    select model,count(*) as count 
    from temptable 
    where trim(model) in ("A4","A5","A6") 
    group by model 
    having count(*) >= 50
    """).show()

+-----+-----+
|model|count|
+-----+-----+
|   A4| 1381|
|   A5|  882|
|   A6|  748|
+-----+-----+



In [8]:
spark.sql("select count(*) as count_all from temptable").show()

+---------+
|count_all|
+---------+
|    10668|
+---------+



In [9]:
tempdf = spark.sql("""
    select transmission,
    count(*) as count 
    from temptable 
    where transmission in ('Automatic','Manual') 
    group by transmission 
    having count(*) >=50
    order by transmission,count
    """)

In [10]:
tempdf.show()
"""
    slect,from,where,group by,having,order by顺序
"""
print(type(tempdf))

+------------+-----+
|transmission|count|
+------------+-----+
|   Automatic| 2708|
|      Manual| 4369|
+------------+-----+

<class 'pyspark.sql.dataframe.DataFrame'>


In [11]:
mysql_OrderItems = spark.read.format("jdbc")\
    .option("driver","com.mysql.jdbc.Driver")\
    .option("url","jdbc:mysql://139.155.70.177:3306/Gadaite")\
    .option("dbtable","OrderItems")\
    .option("user","root")\
    .option("password","zzjz123")\
    .load()

In [12]:
mysql_OrderItems.createOrReplaceTempView("OrderItems")
spark.sql("select * from OrderItems").show()

+---------+----------+-------+--------+----------+
|order_num|order_item|prod_id|quantity|item_price|
+---------+----------+-------+--------+----------+
|    20005|         1|   BR01|     100|      5.49|
|    20005|         2|   BR03|     100|     10.99|
|    20006|         1|   BR01|      20|      5.99|
|    20006|         2|   BR02|      10|      8.99|
|    20006|         3|   BR03|      10|     11.99|
|    20007|         1|   BR03|      50|     11.49|
|    20007|         2| BNBG01|     100|      2.99|
|    20007|         3| BNBG02|     100|      2.99|
|    20007|         4| BNBG03|     100|      2.99|
|    20007|         5| RGAN01|      50|      4.49|
|    20008|         1| RGAN01|       5|      4.99|
|    20008|         2|   BR03|       5|     11.99|
|    20008|         3| BNBG01|      10|      3.49|
|    20008|         4| BNBG02|      10|      3.49|
|    20008|         5| BNBG03|      10|      3.49|
|    20009|         1| BNBG01|     250|      2.49|
|    20009|         2| BNBG02| 

In [13]:
#查询每个订单号，分组，并统计，排序
spark.sql("""
    select order_num,
    count(*) as order_lines 
    from OrderItems 
    group by order_num 
    order by order_lines
    """).show()

+---------+-----------+
|order_num|order_lines|
+---------+-----------+
|    20005|          2|
|    20006|          3|
|    20009|          3|
|    20007|          5|
|    20008|          5|
+---------+-----------+



In [14]:
mysql_Products = spark.read.format("jdbc")\
    .option("driver","com.mysql.jdbc.Driver")\
    .option("url","jdbc:mysql://139.155.70.177:3306/Gadaite")\
    .option("dbtable","Products")\
    .option("user","root")\
    .option("password","zzjz123")\
    .load()
mysql_Products.createOrReplaceTempView("Products")

In [15]:
spark.sql("""
    select * from Products
    """).show()

+-------+-------+-------------------+----------+--------------------+
|prod_id|vend_id|          prod_name|prod_price|           prod_desc|
+-------+-------+-------------------+----------+--------------------+
| BNBG01|  DLL01|  Fish bean bag toy|      3.49|Fish bean bag toy...|
| BNBG02|  DLL01|  Bird bean bag toy|      3.49|Bird bean bag toy...|
| BNBG03|  DLL01|Rabbit bean bag toy|      3.49|Rabbit bean bag t...|
|   BR01|  BRS01|  8 inch teddy bear|      5.99|8 inch teddy bear...|
|   BR02|  BRS01| 12 inch teddy bear|      8.99|12 inch teddy bea...|
|   BR03|  BRS01| 18 inch teddy bear|     11.99|18 inch teddy bea...|
| RGAN01|  DLL01|        Raggedy Ann|      4.99|18 inch Raggedy A...|
|  RYL01|  FNG01|          King doll|      9.49|12 inch king doll...|
|  RYL02|  FNG01|         Queen doll|      9.49|12 inch queen dol...|
+-------+-------+-------------------+----------+--------------------+



In [16]:
spark.sql("""
    select 
    vend_id,
    min(prod_price) as cheapest_item 
    from Products 
    group by vend_id 
    order by cheapest_item
    """).show()

+-------+-------------+
|vend_id|cheapest_item|
+-------+-------------+
|  DLL01|         3.49|
|  BRS01|         5.99|
|  FNG01|         9.49|
+-------+-------------+



In [17]:
#查询所有订单号，分组，排序
spark.sql("""
    select 
    order_num  
    from OrderItems
    group by order_num 
    having sum(quantity) >=100 
    order by order_num
    """).show()

+---------+
|order_num|
+---------+
|    20005|
|    20007|
|    20009|
+---------+



In [18]:
spark.sql("select * from OrderItems").show()

+---------+----------+-------+--------+----------+
|order_num|order_item|prod_id|quantity|item_price|
+---------+----------+-------+--------+----------+
|    20005|         1|   BR01|     100|      5.49|
|    20005|         2|   BR03|     100|     10.99|
|    20006|         1|   BR01|      20|      5.99|
|    20006|         2|   BR02|      10|      8.99|
|    20006|         3|   BR03|      10|     11.99|
|    20007|         1|   BR03|      50|     11.49|
|    20007|         2| BNBG01|     100|      2.99|
|    20007|         3| BNBG02|     100|      2.99|
|    20007|         4| BNBG03|     100|      2.99|
|    20007|         5| RGAN01|      50|      4.49|
|    20008|         1| RGAN01|       5|      4.99|
|    20008|         2|   BR03|       5|     11.99|
|    20008|         3| BNBG01|      10|      3.49|
|    20008|         4| BNBG02|      10|      3.49|
|    20008|         5| BNBG03|      10|      3.49|
|    20009|         1| BNBG01|     250|      2.49|
|    20009|         2| BNBG02| 

In [19]:
#确定最佳顾客，通过查询总花费
spark.sql("""
    select 
    order_num 
    from OrderItems 
    group by order_num 
    having sum(quantity*item_price) >=1000 
    order by order_num
    """).show()

+---------+
|order_num|
+---------+
|    20005|
|    20007|
|    20009|
+---------+



In [20]:
spark.stop()